<a href="https://colab.research.google.com/github/khanyofficial/inkers1/blob/master/u17_trying%20to%20increase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install imgaug imageio imgaug
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import keras
from google.colab import files, drive
from keras import backend as k
from keras import backend as K
from keras.callbacks import Callback, BaseLogger, ProgbarLogger, ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau, LambdaCallback, LearningRateScheduler
from keras.datasets import cifar10
from keras.layers import Concatenate, Dense, Dropout, Conv2D, ZeroPadding2D, MaxPooling2D, BatchNormalization, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Reshape, Lambda, Conv1D, SeparableConv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, RMSprop, Adadelta
from keras.utils import np_utils
from keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf
import inspect
from keras import backend
from keras import utils
from keras_preprocessing import image
import math 
from keras.metrics import top_k_categorical_accuracy
import functools
import imgaug as ia
from imgaug import augmenters as iaa
import imageio

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

#preprocessing to add files to google drive etc , add topk accuracy etc
LOG_DIR = './log' 

from google.colab import files, drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


ia.seed(1)
images_train = x_train
images_test = x_test

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


seq = iaa.Sequential([
    iaa.Fliplr(0.5), 
    iaa.Crop(percent=(0, 0.1)), 
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.MedianBlur(k=(3, 11)),
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-5, 5),
        shear=(-8, 8)
    ),
    iaa.Add(50, per_channel=True),
    iaa.Sharpen(alpha=0.5),
    iaa.CropAndPad(
    percent=(-0.25, 0.25),
    pad_mode=ia.ALL,
    pad_cval=(0, 128)),
    iaa.Superpixels(p_replace=0.5, n_segments=64),
    iaa.AdditiveGaussianNoise(scale=0.05*255),
    iaa.CoarseDropout(0.02, size_percent=0.5),
    iaa.Invert(0.25, per_channel=0.5),
    iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
], random_order=True) 

images_aug_train = seq.augment_images(images_train)
images_aug_test = seq.augment_images(images_test)
print (images_aug_train.shape, images_aug_test.shape)

170500096/170498071 [==============================] - 57s 0us/step


In [3]:

# Hyperparameters
batchsize = 16
num_classes = 10
epochs = 5
l = 40
num_filter = 12
compression = 0.5
#dropout_rate = 0.2

nb_train_samples = 50000 # 3000 training samples
nb_valid_samples = 50000 # 100 validation samples
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_dim_ordering() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])
 
      
    return X_train, Y_train, X_valid, Y_valid
  
# Load CIFAR10 Data
x_train1, y_train1, x_test1, y_test1 = load_cifar10_data(36,36)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
x_train1 = x_train1[:,4:36,4:36,:]
x_test1 = x_test1[:,4:36,4:36,:]
x_train = np.reshape(np.append(x_train, x_train1), (100000, 32,32,3))
x_test = np.reshape(np.append(x_test, x_test1), (20000, 32,32,3))

x_train = np.reshape(np.append(x_train, images_aug_train), (150000, 32,32,3))
x_test = np.reshape(np.append(x_test, images_aug_test), (30000, 32,32,3))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert to one hot encoing
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train1 = keras.utils.to_categorical(y_train1, num_classes)
y_test1 = keras.utils.to_categorical(y_test1, num_classes)
y_train = np.reshape(np.append(y_train, y_train1), (100000, 10))
y_test = np.reshape(np.append(y_test, y_test1), (20000, 10))
y_train = np.reshape(np.append(y_train, y_train1), (150000, 10))
y_test = np.reshape(np.append(y_test, y_test1), (30000, 10))
x_train = x_train[:,4:28,4:28,:]
x_test = x_test[:,4:28,4:28,:]
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

checkpoints = []
checkpoints.append(ModelCheckpoint("savedfile.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1))


(150000, 24, 24, 3) (30000, 24, 24, 3) (150000, 10) (30000, 10)


In [4]:



"""Utilities for real-time data augmentation on image data.
"""
print("overriding keras preprocessing image.py file to add the below features")
print("to apply contrast_stretching, adaptive_equalization,  histogram_equalization augmentation")
random_rotation = image.random_rotation
random_shift = image.random_shift
random_shear = image.random_shear
random_zoom = image.random_zoom
apply_channel_shift = image.apply_channel_shift
random_channel_shift = image.random_channel_shift
apply_brightness_shift = image.apply_brightness_shift
random_brightness = image.random_brightness
apply_affine_transform = image.apply_affine_transform
load_img = image.load_img


def array_to_img(x, data_format=None, scale=True, dtype=None):
    if data_format is None:
        data_format = backend.image_data_format()
    if 'dtype' in inspect.getargspec(image.array_to_img).args:
        if dtype is None:
            dtype = backend.floatx()
        return image.array_to_img(x,
                                  data_format=data_format,
                                  scale=scale,
                                  dtype=dtype)
    return image.array_to_img(x,
                              data_format=data_format,
                              scale=scale)


def img_to_array(img, data_format=None, dtype=None):
    if data_format is None:
        data_format = backend.image_data_format()
    if 'dtype' in inspect.getargspec(image.img_to_array).args:
        if dtype is None:
            dtype = backend.floatx()
        return image.img_to_array(img, data_format=data_format, dtype=dtype)
    return image.img_to_array(img, data_format=data_format)


def save_img(path,
             x,
             data_format=None,
             file_format=None,
             scale=True, **kwargs):
    if data_format is None:
        data_format = backend.image_data_format()
    return image.save_img(path,
                          x,
                          data_format=data_format,
                          file_format=file_format,
                          scale=scale, **kwargs)


class Iterator(image.Iterator, utils.Sequence):
   
    pass


class DirectoryIterator(image.DirectoryIterator, Iterator):  

    def __init__(self, directory, image_data_generator,
                 target_size=(256, 256),
                 color_mode='rgb',
                 classes=None,
                 class_mode='categorical',
                 batch_size=32,
                 shuffle=True,
                 seed=None,
                 data_format=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='png',
                 follow_links=False,
                 subset=None,
                 interpolation='nearest',
                 dtype=None):
        if data_format is None:
            data_format = backend.image_data_format()
        kwargs = {}
        if 'dtype' in inspect.getargspec(
                image.ImageDataGenerator.__init__).args:
            if dtype is None:
                dtype = backend.floatx()
            kwargs['dtype'] = dtype
        super(DirectoryIterator, self).__init__(
            directory, image_data_generator,
            target_size=target_size,
            color_mode=color_mode,
            classes=classes,
            class_mode=class_mode,
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            data_format=data_format,
            save_to_dir=save_to_dir,
            save_prefix=save_prefix,
            save_format=save_format,
            follow_links=follow_links,
            subset=subset,
            interpolation=interpolation,
            **kwargs)


class NumpyArrayIterator(image.NumpyArrayIterator, Iterator):

    def __init__(self, x, y, image_data_generator,
                 batch_size=32,
                 shuffle=False,
                 sample_weight=None,
                 seed=None,
                 data_format=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='png',
                 subset=None,
                 dtype=None):
        if data_format is None:
            data_format = backend.image_data_format()
        kwargs = {}
        if 'dtype' in inspect.getargspec(
                image.NumpyArrayIterator.__init__).args:
            if dtype is None:
                dtype = backend.floatx()
            kwargs['dtype'] = dtype
        super(NumpyArrayIterator, self).__init__(
            x, y, image_data_generator,
            batch_size=batch_size,
            shuffle=shuffle,
            sample_weight=sample_weight,
            seed=seed,
            data_format=data_format,
            save_to_dir=save_to_dir,
            save_prefix=save_prefix,
            save_format=save_format,
            subset=subset,
            **kwargs)


class ImageDataGenerator(image.ImageDataGenerator):
    def __init__(self,
                 featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 zca_epsilon=1e-6,
                 rotation_range=0,
                 width_shift_range=0.,
                 height_shift_range=0.,
                 brightness_range=None,
                 shear_range=0.,
                 zoom_range=0.,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=False,
                 vertical_flip=False,
                 rescale=None,
                 preprocessing_function=None,
                 data_format=None,
                 validation_split=0.0,
                 dtype=None,
                 contrast_stretching=True, 
                 adaptive_equalization=True,  
                 histogram_equalization=True):
        if data_format is None:
            self.counter = 0
            data_format = backend.image_data_format()
            self.contrast_stretching = contrast_stretching #####
            self.adaptive_equalization = adaptive_equalization #####
            self.histogram_equalization = histogram_equalization #####
        kwargs = {}
        if 'dtype' in inspect.getargspec(
                image.ImageDataGenerator.__init__).args:
            if dtype is None:
                dtype = backend.floatx()
            kwargs['dtype'] = dtype
        super(ImageDataGenerator, self).__init__(
            featurewise_center=featurewise_center,
            samplewise_center=samplewise_center,
            featurewise_std_normalization=featurewise_std_normalization,
            samplewise_std_normalization=samplewise_std_normalization,
            zca_whitening=zca_whitening,
            zca_epsilon=zca_epsilon,
            rotation_range=rotation_range,
            width_shift_range=width_shift_range,
            height_shift_range=height_shift_range,
            brightness_range=brightness_range,
            shear_range=shear_range,
            zoom_range=zoom_range,
            channel_shift_range=channel_shift_range,
            fill_mode=fill_mode,
            cval=cval,
            horizontal_flip=horizontal_flip,
            vertical_flip=vertical_flip,
            rescale=rescale,
            preprocessing_function=preprocessing_function,
            data_format=data_format,
            validation_split=validation_split,
            **kwargs)


array_to_img.__doc__ = image.array_to_img.__doc__
img_to_array.__doc__ = image.img_to_array.__doc__
save_img.__doc__ = image.save_img.__doc__

overriding keras preprocessing image.py file to add the below features
to apply contrast_stretching, adaptive_equalization,  histogram_equalization augmentation


In [5]:
print("helper funtion to apply contrast_stretching, adaptive_equalization,  histogram_equalization augmentation")

def random_transform(self, x):
  img_row_axis = self.row_axis - 1
  img_col_axis = self.col_axis - 1
  img_channel_axis = self.channel_axis - 1
# use composition of homographies
# to generate final transform that needs to be applied
  if self.rotation_range:
     theta = np.pi / 180 * np.random.uniform(-self.rotation_range, self.rotation_range)
  else:
     theta = 0
  if self.height_shift_range:
     tx = np.random.uniform(-self.height_shift_range, self.height_shift_range) * x.shape[img_row_axis]
  else:
     tx = 0
  if self.width_shift_range:
     ty = np.random.uniform(-self.width_shift_range, self.width_shift_range) * x.shape[img_col_axis]
  else:
     ty = 0
  if self.shear_range:
     shear = np.random.uniform(-self.shear_range, self.shear_range)
  else:
     shear = 0
  if self.zoom_range[0] == 1 and self.zoom_range[1] == 1:
     zx, zy = 1, 1
  else:
     zx, zy = np.random.uniform(self.zoom_range[0], self.zoom_range[1], 2)
  transform_matrix = None
  if theta != 0:
      rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                        [np.sin(theta), np.cos(theta), 0],
                                        [0, 0, 1]])
      transform_matrix = rotation_matrix
  if tx != 0 or ty != 0:
    shift_matrix = np.array([[1, 0, tx],
                                     [0, 1, ty],
                                     [0, 0, 1]])
    transform_matrix = shift_matrix if transform_matrix is None else np.dot(transform_matrix, shift_matrix)
  if shear != 0:
     shear_matrix = np.array([[1, -np.sin(shear), 0],
                                    [0, np.cos(shear), 0],
                                    [0, 0, 1]])
     transform_matrix = shear_matrix if transform_matrix is None else np.dot(transform_matrix, shear_matrix)
  if zx != 1 or zy != 1:
     zoom_matrix = np.array([[zx, 0, 0],
                                    [0, zy, 0],
                                    [0, 0, 1]])
     transform_matrix = zoom_matrix if transform_matrix is None else np.dot(transform_matrix, zoom_matrix)
  if transform_matrix is not None:
     h, w = x.shape[img_row_axis], x.shape[img_col_axis]
     transform_matrix = transform_matrix_offset_center(transform_matrix, h, w)
     x = apply_transform(x, transform_matrix, img_channel_axis,
                                fill_mode=self.fill_mode, cval=self.cval)
  if self.channel_shift_range != 0:
     x = random_channel_shift(x, self.channel_shift_range, img_channel_axis)
  if self.horizontal_flip:
     if np.random.random() < 0.5:
        x = flip_axis(x, img_col_axis)
  if self.vertical_flip:
     if np.random.random() < 0.5:
        x = flip_axis(x, img_row_axis)
                
  if self.contrast_stretching: #####
     if np.random.random() < 0.5: #####
        p2, p98 = np.percentile(x, (2, 98)) #####
        x = exposure.rescale_intensity(x, in_range=(p2, p98)) #####
 
  if self.adaptive_equalization: #####
     if np.random.random() < 0.5: #####
        x = exposure.equalize_adapthist(x, clip_limit=0.03) #####
                
  if self.histogram_equalization: #####
     if np.random.random() < 0.5: #####
        x = exposure.equalize_hist(x) #####
                
  return x

helper funtion to apply contrast_stretching, adaptive_equalization,  histogram_equalization augmentation


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
      inter_channel = num_filter * 4 
      #temp = ZeroPadding2D((3, 3))(temp)
      BatchNorm = BatchNormalization(epsilon=1.1e-5)(temp)
      relu = Activation('relu')(BatchNorm)
      Cnv1D = SeparableConv2D(inter_channel, 1, 1,  bias=False)(relu)
      if dropout_rate>0:
        Cnv1D = Dropout(dropout_rate)(Cnv1D)
      
      BN2 = BatchNormalization(epsilon=1.1e-5)(Cnv1D)
      relu2 = Activation('relu')(BN2)
      #zp2 = ZeroPadding2D((1, 1), name='x2_zeropadding')(relu2)
      Conv2D_3_3 = SeparableConv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu2)
      #zp3 = ZeroPadding2D((1, 1), name='x2_zeropadding')(Conv2D_3_3)
      concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
      temp = concat
      
        
    return temp

def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = SeparableConv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg


def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [7]:
num_filter = 32
dropout_rate = 0.2
l = 10
compression = 0.9
print(compression)
input = Input(shape=(img_height, img_width, channel,))

x = SeparableConv2D(32, (7,7), use_bias=False ,padding='same', strides=(2, 2))(input)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = SeparableConv2D(32, 3, 3, subsample=(2, 2), bias=False)(input)
x = BatchNormalization(epsilon=1.1e-5)(x)
First_Conv2D = Activation('relu')(x)
First_Conv2D = ZeroPadding2D((1, 1))(First_Conv2D)
First_Conv2D = MaxPooling2D((3, 3), strides=(2, 2))(First_Conv2D)

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = SeparableConv2D(num_filter, (5, 5), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)


Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model = Model(inputs=[input], outputs=[output])
model.summary()

0.9


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (3, 3), strides=(2, 2), use_bias=False)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(128, (1, 1), use_bias=False)`
  if __name__ == '__main__':


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d_3 (SeparableCo (None, 24, 24, 32)   171         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 24, 24, 32)   128         separable_conv2d_3[0][0]         
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 24, 24, 32)   0           batch_normalization_2[0][0]      
__________________________________________________________________________________________________
separable_

In [6]:
datagen  = ImageDataGenerator(rotation_range=2.5, width_shift_range=5./32, height_shift_range=5./32, 
                              contrast_stretching=True, adaptive_equalization=True,  histogram_equalization=True)

datagen.fit(x_train)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:179: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()


In [10]:
print("epoch 1-5")

def step_decay(epoch):
  initial_lrate = 0.2
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])
# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2*len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()



epoch 1-5
Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00001: LearningRateScheduler reducing learning rate to 0.2.
9375/9375 [==============================] - 2086s 223ms/step - loss: 2.0205 - acc: 0.2539 - top_k_categorical_accuracy: 0.7601 - val_loss: 1.9634 - val_acc: 0.2839 - val_top_k_categorical_accuracy: 0.7907

Epoch 00001: val_acc improved from -inf to 0.28390, saving model to savedfile.h5
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.2.
9375/9375 [==============================] - 2053s 219ms/step - loss: 1.8993 - acc: 0.3055 - top_k_categorical_accuracy: 0.7960 - val_loss: 1.8950 - val_acc: 0.3442 - val_top_k_categorical_accuracy: 0.8084

Epoch 00002: val_acc improved from 0.28390 to 0.34420, saving model to savedfile.h5
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00003: LearningRateScheduler reducing l

In [11]:
print("epoch 6-10")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.125
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()
                    

epoch 6-10
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00001: LearningRateScheduler reducing learning rate to 0.125.
9375/9375 [==============================] - 2121s 226ms/step - loss: 1.5339 - acc: 0.4500 - top_k_categorical_accuracy: 0.8563 - val_loss: 1.6846 - val_acc: 0.4554 - val_top_k_categorical_accuracy: 0.8535

Epoch 00001: val_acc improved from 0.41650 to 0.45537, saving model to savedfile.h5
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.125.
9375/9375 [==============================] - 2103s 224ms/step - loss: 1.4851 - acc: 0.4661 - top_k_categorical_accuracy: 0.8638 - val_loss: 1.5602 - val_acc: 0.4739 - val_top_k_categorical_accuracy: 0.8579

Epoch 00002: val_acc improved from 0.45537 to 0.47393, saving model to

In [12]:
print("epoch 11-15")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")


def step_decay(epoch):
  initial_lrate = 0.1
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

epoch 11-15
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
9375/9375 [==============================] - 2142s 228ms/step - loss: 1.3383 - acc: 0.5202 - top_k_categorical_accuracy: 0.8819 - val_loss: 1.3196 - val_acc: 0.5422 - val_top_k_categorical_accuracy: 0.8822

Epoch 00001: val_acc improved from 0.53380 to 0.54220, saving model to savedfile.h5
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.2.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
9375/9375 [==============================] - 2110s 225ms/step - loss: 1.3124 - acc: 0.5289 - top_k_categorical_accuracy: 0.8856 - val_loss: 1.3824 - val_acc: 0.5337 - val_top_k_categorical_accuracy: 0.8747

Epoch 00002: val_acc did not improve from 0.54220
Epoch 3/5

Epoch 00003:

In [10]:
print("epoch 16-20")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")


def step_decay(epoch):
  initial_lrate = 0.05
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

epoch 16-20
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.05.

Epoch 00001: LearningRateScheduler reducing learning rate to 0.05.
9375/9375 [==============================] - 1967s 210ms/step - loss: 1.1933 - acc: 0.5720 - top_k_categorical_accuracy: 0.8985 - val_loss: 1.1884 - val_acc: 0.5920 - val_top_k_categorical_accuracy: 0.8907

Epoch 00001: val_acc improved from -inf to 0.59203, saving model to savedfile.h5
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.05.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.05.
9375/9375 [==============================] - 1975s 211ms/step - loss: 1.1776 - acc: 0.5792 - top_k_categorical_accuracy: 0.9005 - val_loss: 1.2626 - val_acc: 0.5768 - val_top_k_categorical_accuracy: 0.8928

Epoch 00002: val_acc did not improve from 0.59203
Epoch 3/5

Epoch 00003

In [1]:
print("epoch 21-25")
print (" changing back lr to 0.05 and training on base data set with no augmentation for epoch 21-25 as I am very stuck with 0.2% acc per epoch")
print (" based on the results I may re-add the augmentation. ")


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train[:,4:28,4:28,:]
x_test = x_test[:,4:28,4:28,:]
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.001
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 3 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

epoch 21-25
 changing back lr to 0.05 and training on base data set with no augmentation for epoch 21-25 as I am very stuck with 0.2% acc per epoch
 based on the results I may re-add the augmentation. 


NameError: ignored

In [0]:
print("epoch 26-30")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")


def step_decay(epoch):
  initial_lrate = 0.025
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

In [0]:
print("epoch 31-35")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")


def step_decay(epoch):
  initial_lrate = 0.05
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

In [0]:
print("epoch 36-40")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")


def step_decay(epoch):
  initial_lrate = 0.05
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

In [0]:
print("epoch 41-45")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")


def step_decay(epoch):
  initial_lrate = 0.05
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

In [0]:
print("epoch 46-50")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.05
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy'])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= 2 * len(x_train) / 32, epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

In [0]:
print("epoch 51-100")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.025
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy', topk16, topk32, topk64])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= int(1.5 * len(x_train) / 32), epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()

In [0]:
print("epoch 100-150")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.005
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy', topk16, topk32, topk64])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= int(1.5 * len(x_train) / 32), epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()


In [0]:
print("epoch 150-200")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.0125
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy', topk16, topk32, topk64])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= int(1.5 * len(x_train) / 32), epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()


In [0]:
print("epoch 200-250")

from google.colab import files, drive
drive.mount('/content/drive')
model = load_model("/content/drive/My Drive/eip/savedmodel.h5")
model.load_weights("/content/drive/My Drive/eip/savedweights.h5")

def step_decay(epoch):
  initial_lrate = 0.0125
  drop = 0.5
  epochs_drop = 10
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrs = LearningRateScheduler(step_decay, verbose=1)
checkpoints.append(lrs)

# determine Loss function and Optimizer
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy','top_k_categorical_accuracy', topk16, topk32, topk64])

# determine Loss function and Optimizer
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=checkpoints)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16), steps_per_epoch= int(1.5 * len(x_train) / 32), epochs=epochs , validation_data=(x_test, y_test), verbose = 1, callbacks=checkpoints)

from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
checkpoints.pop()


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the trained weights in to .h5 format
from google.colab import files, drive
drive.mount('/content/drive')
model.save("/content/drive/My Drive/eip/savedmodel.h5")
model.save_weights("/content/drive/My Drive/eip/savedweights.h5")
model.save("savedmodel.h5")
model.save_weights("savedweights.h5")
